In [1]:
from glob import glob
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from segmentation_models import Unet
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import cv2
import keras
import random


num_classes = 4
train_percent = .8
imgScale = .5

# input image dimensions
img_rows, img_cols = 256, 1600

# set paths to train and test image datasets
TRAIN_PATH = 'train_images/'
TEST_PATH = 'test_images/'

# load dataframe with train labels
train_df = pd.read_csv('train.csv')
train_df = train_df.reindex(np.random.permutation(train_df.index))


Using TensorFlow backend.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

Segmentation Models: using `keras` framework.


In [2]:
train_fns = sorted(glob(TRAIN_PATH + '*.jpg'))
test_fns = sorted(glob(TEST_PATH + '*.jpg'))

print('There are {} images in the train set.'.format(len(train_fns)))
print('There are {} images in the test set.'.format(len(test_fns)))

train_df.head(10)

# split column
split_df = train_df["ImageId_ClassId"].str.split("_", n = 1, expand = True)

# add new columns to train_df
train_df['Image'] = split_df[0]
train_df['Label'] = split_df[1]

# check the result
train_df.head()

defect1 = train_df[train_df['Label'] == '1'].EncodedPixels.count()
defect2 = train_df[train_df['Label'] == '2'].EncodedPixels.count()
defect3 = train_df[train_df['Label'] == '3'].EncodedPixels.count()
defect4 = train_df[train_df['Label'] == '4'].EncodedPixels.count()
print([defect1, defect2, defect3, defect4])
min_count = min([defect1, defect2, defect3, defect4])

path = TRAIN_PATH
count_one = 0;
count_two = 0;
count_three = 0;
count_four = 0;
count_one_test = 0;
count_two_test = 0;
count_three_test = 0;
count_four_test = 0;


x_train = []
y_train = []
x_test = []
y_test =[]

train_count = int(min_count*train_percent)
test_count = int((1-train_percent)*min_count)
images = sorted(glob(path + '*.jpg'))

for im in range(0, len(images)):
    label = int(train_df.iloc[im][3])-1

    orgimage = cv2.imread(path+train_df.iloc[im][2])
    newX,newY = orgimage.shape[1]*imgScale, orgimage.shape[0]*imgScale
    image = cv2.resize(orgimage,(int(newX),int(newY)))

    if(train_df.iloc[im][1]) != 'nan':
        if(label == 0):
            if(count_one <= train_count):
                count_one+=1
                x_train.append(image)
                y_train.append(label)
            elif(count_one_test <= test_count):
                count_one_test+=1
                x_test.append(image)
                y_test.append(label)
        if(label == 1):
            if(count_two <= train_count):
                count_two+=1
                x_train.append(image)
                y_train.append(label)
            elif(count_two_test <= test_count):
                count_two_test+=1
                x_test.append(image)
                y_test.append(label)
        if(label == 2):
            if(count_three <= train_count):
                count_three+=1
                x_train.append(image)
                y_train.append(label)
            elif(count_three_test <= test_count):
                count_three_test+=1
                x_test.append(image)
                y_test.append(label)
        if(label == 3):
            if(count_four <= train_count):
                count_four+=1
                x_train.append(image)
                y_train.append(label)
            elif(count_four_test <= test_count):
                count_four_test+=1
                x_test.append(image)
                y_test.append(label)

    #print(len(y_test), len(x_test), len(y_train), len(x_train))
    #print(count_four_test,count_three_test,count_two_test,count_one_test, test_count)
    all_data_count = test_count
    if(count_four_test >= all_data_count and
       count_three_test >= all_data_count and
       count_two_test >= all_data_count and
       count_one_test >= all_data_count):
        break;


There are 12568 images in the train set.
There are 1801 images in the test set.
[897, 247, 5150, 801]


In [3]:
print(y_test)

[3, 3, 3, 2, 2, 3, 3, 2, 2, 2, 3, 3, 3, 3, 0, 2, 3, 0, 3, 0, 3, 2, 0, 2, 0, 0, 2, 0, 3, 2, 3, 0, 2, 0, 3, 2, 2, 2, 3, 3, 0, 3, 2, 3, 3, 2, 0, 3, 3, 2, 0, 2, 0, 2, 0, 3, 3, 0, 0, 0, 0, 3, 3, 3, 0, 2, 2, 2, 2, 0, 2, 0, 0, 3, 3, 3, 3, 3, 2, 0, 2, 2, 2, 0, 0, 3, 3, 3, 2, 3, 0, 3, 3, 2, 2, 3, 1, 3, 1, 3, 0, 0, 0, 0, 3, 1, 0, 3, 0, 2, 3, 3, 3, 1, 3, 0, 3, 0, 1, 1, 1, 2, 1, 0, 0, 0, 2, 2, 3, 3, 0, 2, 0, 2, 0, 2, 1, 2, 0, 0, 2, 0, 0, 0, 0, 0, 2, 1, 0, 1, 0, 0, 1, 2, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [4]:
y_train_orig = y_train
y_test_orig = y_test
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
x_test = np.asarray(x_test)
y_test = np.asarray(y_test)
y_train = keras.utils.to_categorical(y_train,4)
y_test = keras.utils.to_categorical(y_test,4)

In [5]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5,5), strides=(1, 1),
                 activation='relu',
                 input_shape=(image.shape[0],image.shape[1],3)))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2))) 
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(4, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

#datagen = ImageDataGenerator(
#    featurewise_center=True,
#    featurewise_std_normalization=True,
#    rotation_range=20,
#    width_shift_range=0.2,
#    height_shift_range=0.2,
#    horizontal_flip=True)

model.fit(x_train, y_train,
          batch_size=16,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))

#datagen.fit(x_train)

#model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), validation_data=(x_test, y_test), steps_per_epoch=len(x_train),
#                    epochs=epochs)






Train on 792 samples, validate on 199 samples
Epoch 1/10
792/792 [==============================] - 120s 151ms/step - loss: 215.5112 - accuracy: 0.2487 - val_loss: 1.3965 - val_accuracy: 0.2412
Epoch 2/10
792/792 [==============================] - 110s 139ms/step - loss: 1.3707 - accuracy: 0.2917 - val_loss: 1.4242 - val_accuracy: 0.2211
Epoch 3/10
792/792 [==============================] - 106s 133ms/step - loss: 1.3739 - accuracy: 0.3081 - val_loss: 1.4025 - val_accuracy: 0.2462
Epoch 4/10
792/792 [==============================] - 105s 132ms/step - loss: 1.3406 - accuracy: 0.3270 - val_loss: 1.4816 - val_accuracy: 0.2513
Epoch 5/10
792/792 [==============================] - 108s 136ms/step - loss: 1.3039 - accuracy: 0.3460 - val_loss: 1.5757 - val_accuracy: 0.2462
Epoch 6/10
792/792 [==============================] - 112s 141ms/step - loss: 1.2417 - accuracy: 0.3396 - val_loss: 1.5725 - val_accuracy: 0.2060
Epoch 7/10
792/792 [==============================] - 108s 136ms/step - lo

In [6]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

y_pred = model.predict_classes(x_test, verbose=1)
print(y_pred)

print(y_test_orig)
print(y_test)


Test loss: 2.040714062638019
Test accuracy: 0.24120603501796722
199/199 [==============================] - 4s 22ms/step
[3 3 3 3 3 2 3 3 3 3 3 3 1 0 2 3 3 3 3 3 3 3 3 1 3 3 2 3 3 0 3 1 3 3 2 3 3
 3 3 3 2 3 2 3 3 0 3 3 2 3 3 3 1 3 3 3 0 3 3 0 3 3 3 3 0 3 0 3 3 3 3 3 3 3
 3 3 3 3 2 3 3 3 3 0 3 1 1 3 3 3 3 3 3 3 0 3 3 2 1 0 3 3 3 3 3 2 3 3 3 3 3
 1 3 3 3 3 1 3 0 3 1 3 0 2 1 3 3 3 2 1 3 3 3 3 0 3 0 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 1 0 3 3 3 3 3 3 3 3 0 3 3 3 3 1 2 2 3 3 3 3 3 3 3 0 3 3 3 3 2 3 1 2
 3 3 3 3 2 3 3 3 3 3 3 3 1 0]
[3, 3, 3, 2, 2, 3, 3, 2, 2, 2, 3, 3, 3, 3, 0, 2, 3, 0, 3, 0, 3, 2, 0, 2, 0, 0, 2, 0, 3, 2, 3, 0, 2, 0, 3, 2, 2, 2, 3, 3, 0, 3, 2, 3, 3, 2, 0, 3, 3, 2, 0, 2, 0, 2, 0, 3, 3, 0, 0, 0, 0, 3, 3, 3, 0, 2, 2, 2, 2, 0, 2, 0, 0, 3, 3, 3, 3, 3, 2, 0, 2, 2, 2, 0, 0, 3, 3, 3, 2, 3, 0, 3, 3, 2, 2, 3, 1, 3, 1, 3, 0, 0, 0, 0, 3, 1, 0, 3, 0, 2, 3, 3, 3, 1, 3, 0, 3, 0, 1, 1, 1, 2, 1, 0, 0, 0, 2, 2, 3, 3, 0, 2, 0, 2, 0, 2, 1, 2, 0, 0, 2, 0, 0, 0, 0, 0, 2, 1, 0, 1, 0, 0, 1, 2, 2, 1, 1, 1, 